In [28]:
from BCBio import GFF
import operator
from Bio.SeqRecord import SeqRecord
import glob, os
from Bio import SeqIO
import time
import pysam

import gffutils
import pyfaidx
from pysam import VariantFile


In [52]:
sourcebase= '/mnt/test_data/hepavac34/rna_benign/spladdrout'
#sourcebase = '/tmp/'
reffile = '/mnt/test_data/refs/GRCh37.primary_assembly.genome.fa'
fileending = 'gff3'
vcffile= '/tmp/vcf.vcf.gz' #must be indexed!

#db = gffutils.create_db('/tmp/ensg228794.gff', ':memory:')
fasta = pyfaidx.Fasta('/mnt/test_data/refs/GRCh37.primary_assembly.genome.fa')
vcffile='/tmp/vcf.vcf.gz'

outfile = '/tmp/predicted_exons.fa'
outfile2 = '/tmp/predicted_genes.fa'  
vcfrecords=VariantFile(vcffile)


In [30]:
#Getting all gff-files in base directory
def get_gff_files(sourcebase):
    print "Checking for GFF Files in directory..."
    gfffiles = []
    os.chdir(sourcebase)
    for file in glob.glob("*.%s" %fileending):
        gfffiles.append(file)
    print "Found %i file(s) in %s ending with %s." %(len(gfffiles), sourcebase, fileending)
    return gfffiles

In [59]:
def get_gene_sequences(gene):
    sequences = []
    num_mod=0
    for i in db.children(gene,featuretype='exon'):
        #print ("%i %i")%(i.start,i.stop)
        #print i.sequence(fasta)
        events=check_feature_for_vcfevent(i)
        if len(events) == 0:
            sequences.append(SeqIO.SeqRecord(Seq.translate(i.sequence(fasta),to_stop=True),id=i.id,name=gene.id))
        else:
            num_mod+=1
            sequences.append(SeqIO.SeqRecord(get_modified_sequence(i.sequence(fasta),events,i.start),id=i.id,name=gene.id))
            #return get_modified_sequence(i.sequence(fasta),events)
    return sequences , num_mod 

In [32]:
def check_feature_for_vcfevent(feature):
    vcfevents = []
    if feature.chrom not in vcfrecords.header.contigs:
        print "Chrom not found"
        return vcfevents
    for rec in vcfrecords.fetch(feature.chrom,feature.start,feature.stop):
        vcfevents.append(rec)
       #a=rec
    if len(vcfevents) == 0:
        return vcfevents
        #print "no events in range detected (%s - %i on %i)"%(feature.chrom,feature.start,feature.stop)
    else:
        print "%i events in range detected (%s - %i on %i)"%(len(vcfevents),feature.chrom,feature.start,feature.stop)
        
    return vcfevents

In [33]:
def get_modified_sequence(in_seq,events,offset):
    print "Personalisizing sequence len:%i with %i events"%(len(in_seq),len(events))
    lengthchange=0
    for event in events:
        eventpos=event.start-offset
        print "Eventstart %i, offset: %i -> pos:%i"%(event.start,offset,eventpos)
        print "Char at  %i: %s (region +-3: %s)" %(eventpos,in_seq[eventpos],in_seq[eventpos-3:eventpos+3])
        alleles=event.alleles
        l = list(in_seq)
        
        if is_snp(alleles):
          #  print "Found SNP substituting %s with %s" %(l[eventpos],alleles[1])
            if l[eventpos]!= alleles[0]:
                print "warning SNP missmatch"
            l[eventpos]=alleles[1]
        if is_del(alleles):
            l[eventpos]=alleles[1] #TODO falsch!
            lengthchange-=len(alleles[1])-len(alleles[0])
            print "Found deletion: deleting %s from sequences"%alleles[0]
            print alleles
            print event
        if is_insert(alleles):
            l.insert(eventpos,alleles[1])
            lengthchange+=len(alleles[0])-len(alleles[1])
            print "found insert: inserting %s into sequence at pos %i"%alleles[1],eventpos
            print event
    new_seq=''.join(l)
    if (len(in_seq)-len(new_seq)) != lengthchange:
        print "Warning: length changed %i (should be %i)" %(len(in_seq)-len(new_seq),lengthchange)
    return new_seq 

In [34]:
def is_snp(e):
    if len(e[0]) == 1 and len(e[1]) == 1:
        return True
    return False

def is_insert(e):
    if len(e[0])<len(e[1]):
        return True
    return False

def is_del(e):
    if len(e[0])>len(e[1]):
        return True
    return False

In [51]:
def run():
    gfffiles = get_gff_files(sourcebase)
    
    for infile in gfffiles:
        total_genes=0
        total_exons=0
        total_genes_wrote=0
        total_exons_wrote=0
        total_mod=0
        db = gffutils.create_db(infile, ':memory:')
        for gene in db.features_of_type('gene'):
            total_genes+=1
            seqs,num_mod=get_gene_sequences(gene)
            total_exons+=len(seqs)
            total_mod+=num_mod
        total_exons_wrote+=write_records(seqs,'exons')
       # total_genes_wrote+=write_records(seqs,'genes')
        print "processed (and wrote): %s with %i(%i) genes and %i(%i) exons (%i modified))" %(infile,total_genes,total_genes_wrote,total_exons,total_exons_wrote,total_mod)
        
    #ref_recs = load_reffile(reffile)
    
     #   records, records_until_stop = get_records(infile,ref_recs)
      #  write_records(records,records_until_stop,os.path.basename(infile))
        

In [44]:
def write_records(records,type):
    #translate records
    
    if type == 'exons':
        with open(outfile, "w") as out_handle:
            return SeqIO.write(records, out_handle, "fasta")
    if type == 'genes':
        #s=[]
        #for seq in records:
        #    s.append(str(seq.seq))
        #gene=SeqIO.SeqRecord(''.join(s),id=s[0].id)
        print "gene wrting goes here"
       # with open(outfile, "w") as out_handle:
        #    return SeqIO.write(gene, out_handle, "fasta")    
 

In [60]:
run()

Checking for GFF Files in directory...
Found 5 file(s) in /mnt/test_data/hepavac34/rna_benign/spladdrout ending with gff3.
processed (and wrote): merge_graphs_intron_retention_C3.confirmed.gff3 with 5183(0) genes and 0(0) exons (0 modified))
processed (and wrote): merge_graphs_alt_5prime_C3.confirmed.gff3 with 4847(0) genes and 0(0) exons (0 modified))
Chrom not found
Chrom not found
Chrom not found
Chrom not found


/home/centos/miniconda3/envs/py27/lib/python2.7/site-packages/Bio/Seq.py:2309: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning)


1 events in range detected (chr1 - 169509532 on 169512229)
Personalisizing sequence len:2698 with 1 events
Eventstart 169510649, offset: 169509532 -> pos:1117
Char at  1117: A (region +-3: TAAATC)
warning SNP missmatch
1 events in range detected (chr1 - 169509532 on 169512352)
Personalisizing sequence len:2821 with 1 events
Eventstart 169510649, offset: 169509532 -> pos:1117
Char at  1117: A (region +-3: AAAAAA)
warning SNP missmatch
1 events in range detected (chr14 - 105221967 on 105223262)
Personalisizing sequence len:1296 with 1 events
Eventstart 105221961, offset: 105221967 -> pos:-6
Char at  -6: T (region +-3: GCTTGG)
Found deletion: deleting CCCAGAGAAGACCAA from sequences
('CCCAGAGAAGACCAA', 'C')
chr14	105221962	.	CCCAGAGAAGACCAA	C	.	PASS	ANN=C|frameshift_variant&splice_acceptor_variant&splice_region_variant&intron_variant|HIGH|SIVA1|ENSG00000184990|transcript|ENST00000556195|protein_coding|2/4|c.170-4_179delCCAGAGAAGACCAA|p.Glu57fs||170/579|57/192||WARNING_TRANSCRIPT_NO_START_C

TypeError: SeqRecord (id=exon_28117) has an invalid sequence.

In [58]:
from Bio import Seq